## SaaS Metrics Definitions and Calculations with SQL



### 1. Annual Recurring Revenue (ARR)

| Category | Detail |
| :--- | :--- |
| **Definition** | The value of the predictable, recurring revenue components of a SaaS business's subscription contracts normalized for one year. Excludes one-time fees and variable usage charges. |
| **SQL Concept** | Calculate the Monthly Recurring Revenue (**MRR**) for each active customer, aggregate the total MRR, and multiply by 12. |
| **Formula** | $$\text{ARR} = \sum (\text{Customer MRR}) \times 12$$ |
| **Data Required** | **Customer ID**, **Subscription Start/End Date**, **Monthly Recurring Revenue (MRR)** or **Annual Contract Value (ACV)**, **One-time fees flag/field** (to filter out non-recurring revenue). |


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW ARR 
USING CSV
OPTIONS (
  path "file:/Workspace/Users/olesia.tankersley@thermofisher.com/Optimization/data/ARR.csv",
  header "true",
  inferSchema "true" );

In [0]:
 %sql
select * from ARR limit 5

customer_id,subscription_start_date,subscription_end_date,monthly_recurring_revenue_mrr,subscription_status,one_time_setup_fee
25,2023-06-26,2023-10-24,384,Cancelled,0
45,2023-05-27,2023-06-06,484,Cancelled,222
62,2024-05-28,2024-09-05,864,Cancelled,0
86,2025-09-19,2025-10-29,450,Cancelled,0
108,2024-09-23,2025-03-22,1833,Cancelled,0


In [0]:
%sql

select 12 *(sum(case when subscription_end_date is not null then monthly_recurring_revenue_mrr else 0 end) - sum(one_time_setup_fee)) as annual_recurring_revenue_ARR
from ARR

annual_recurring_revenue_ARR
41844



### 2. Gross Revenue Retention (GRR)

| Category | Detail |
| :--- | :--- |
| **Definition** | The percentage of revenue retained from existing customers over a specific period, accounting *only* for **downgrades** and **churn**. It *excludes* upsells. |
| **SQL Concept** | Calculate the baseline revenue at the start of the period. Subtract revenue lost from churn and downgrades, then divide by the baseline revenue. |
| **Formula** | $$\text{GRR} = \frac{(\text{Starting ARR} - \text{Churn} - \text{Downgrades})}{\text{Starting ARR}} \times 100$$ |
| **Data Required** | **Customer ID**, **Start-of-Period MRR/ARR** (Baseline Revenue), **Revenue Lost from Downgrades**, **Revenue Lost from Churn** (during the period). |



In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW GRR 
USING CSV
OPTIONS (
  path "file:/Workspace/Users/olesia.tankersley@thermofisher.com/Optimization/data/GRR.csv",
  header "true",
  inferSchema "true" );

In [0]:
 %sql
select * from GRR limit 5

customer_id,start_date,end_date,start_of_period_mrr,end_of_period_mrr,retention_status,revenue_lost_churn_downgrade
1,1/1/2024,2/1/2024,2090,2090.0,Retained,0.0
2,1/1/2024,2/1/2024,2555,2555.0,Retained,0.0
3,1/1/2024,2/1/2024,3904,3904.0,Retained,0.0
4,1/1/2024,2/1/2024,2817,2817.0,Retained,0.0
5,1/1/2024,2/1/2024,1769,1769.0,Retained,0.0


In [0]:
%sql
select distinct retention_status from GRR

retention_status
Downgrade
Churn
Retained


In [0]:
%sql
select round(sum(start_of_period_mrr),0) as starting_MRR
      ,round(sum(end_of_period_mrr),0)   as net_churn_downgrades
      ,round(sum(end_of_period_mrr)
      /sum(start_of_period_mrr),2) as gross_revenue_retention_rate_GRR  
from GRR

starting_MRR,net_churn_downgrades,gross_revenue_retention_rate_GRR
557450,518362.0,0.93


In [0]:
%sql
select retention_status, sum(start_of_period_mrr) as Starting_ARR, round(sum(end_of_period_mrr),0) as Net_Churn_Downgrades, round(sum(end_of_period_mrr)/sum(start_of_period_mrr),2) as GRR  
from GRR
group by retention_status

retention_status,Starting_ARR,Net_Churn_Downgrades,GRR
Downgrade,56106,36740.0,0.65
Churn,19722,0.0,0.0
Retained,481622,481622.0,1.0



### 3. Net Revenue Retention (NRR)

| Category | Detail |
| :--- | :--- |
| **Definition** | The percentage of revenue retained from existing customers, accounting for **downgrades**, **churn**, *and* **upsells**. A value over 100% indicates Net Negative Churn. |
| **SQL Concept** | Calculate the baseline revenue. Add revenue from upsells, subtract revenue lost from churn and downgrades, then divide by the baseline revenue. |
| **Formula** | $$\text{NRR} = \frac{(\text{Starting ARR} - \text{Churn} - \text{Downgrades} + \text{Upsells})}{\text{Starting ARR}} \times 100$$ |
| **Data Required** | **Customer ID**, **Start-of-Period MRR/ARR** (Baseline Revenue), **Revenue Lost from Downgrades**, **Revenue Lost from Churn**, **Revenue Gained from Upsells/Expansions** (during the period). |



In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW NRR 
USING CSV
OPTIONS (
  path "file:/Workspace/Users/olesia.tankersley@thermofisher.com/Optimization/data/NRR.csv",
  header "true",
  inferSchema "true",
  dateFormat "yyyy/MM/dd" );

In [0]:
 %sql
select * from NRR limit 5

customer_id,start_date,end_date,start_of_period_mrr,end_of_period_mrr,revenue_lost_churn_downgrade,revenue_gained_upsell
1,1/1/2025,2/1/2025,2951,2951.0,0.0,0.0
2,1/1/2025,2/1/2025,1165,1165.0,0.0,0.0
3,1/1/2025,2/1/2025,2203,2203.0,0.0,0.0
4,1/1/2025,2/1/2025,3354,2381.34,737.88,0.0
5,1/1/2025,2/1/2025,2387,2387.0,0.0,0.0


In [0]:
%sql
select sum(start_of_period_mrr) as starting_ARR
      ,round(sum(revenue_lost_churn_downgrade),0) as revenue_lost_churn_downgrade
      ,round(sum(revenue_gained_upsell),0) as revenue_upsell
      ,round(sum(start_of_period_mrr) - sum(revenue_lost_churn_downgrade) + sum(revenue_gained_upsell),0) as net_revenue_retention
      ,round((sum(start_of_period_mrr) - sum(revenue_lost_churn_downgrade) + sum(revenue_gained_upsell))/sum(start_of_period_mrr),2) as net_revenue_retention_rate_NRR
from NRR

starting_ARR,revenue_lost_churn_downgrade,revenue_upsell,net_revenue_retention,net_revenue_retention_rate_NRR
685555,50669.0,24973.0,659859.0,0.96




### 4. Renewal Rate

| Category | Detail |
| :--- | :--- |
| **Definition** | The percentage of existing contracts that were successfully renewed at the end of their term. |
| **SQL Concept** | Count the contracts renewed during a specific period and divide by the total number of contracts that were due for renewal in that same period. |
| **Formula** | $$\text{Renewal Rate} = \frac{\text{Number of Renewed Contracts}}{\text{Total Contracts Up for Renewal}} \times 100$$ |
| **Data Required** | **Contract ID**, **Contract End Date**, **Renewal Status** (e.g., 'Renewed', 'Non-Renewed'), **Contract Value** (optional, but useful for value-based rate). |




In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW RR 
USING CSV
OPTIONS (
  path "file:/Workspace/Users/olesia.tankersley@thermofisher.com/Optimization/data/RR.csv",
  header "true",
  inferSchema "true",
  dateFormat "yyyy/MM/dd" );

In [0]:
 %sql
select * from RR limit 5

contract_id,contract_end_date,contract_value_acv,renewal_status
1,3/19/2024,9684,Renewed
2,3/5/2024,5697,Renewed
3,2/24/2024,3725,Renewed
4,3/1/2024,9264,Renewed
5,2/25/2024,6423,Renewed


In [0]:
%sql
select count(*) total_contracts
      ,count(case when renewal_status = 'Renewed' then contract_id else null end) as total_renewals
      ,round(count(case when renewal_status = 'Renewed' then contract_id else null end) / count(*),2) as renewal_rate_RR
from RR

total_contracts,total_renewals,renewal_rate_RR
200,177,0.89



### 5. Customer Churn Rate

| Category | Detail |
| :--- | :--- |
| **Definition** | The percentage of customers who canceled or stopped their subscription over a specific period. |
| **SQL Concept** | Count the customers who churned during the period and divide by the total number of customers at the *start* of the period. |
| **Formula** | $$\text{Customer Churn Rate} = \frac{\text{Customers Churned in Period}}{\text{Customers at Start of Period}} \times 100$$ |
| **Data Required** | **Customer ID**, **Start-of-Period Customer Count** (for the denominator/cohort), **Customer Status** (e.g., 'Active', 'Churned'), **Churn Date**. |




In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW CCR 
USING CSV
OPTIONS (
  path "file:/Workspace/Users/olesia.tankersley@thermofisher.com/Optimization/data/CCR.csv",
  header "true",
  inferSchema "true" ,
  dateFormat "yyyy/MM/dd");

In [0]:
 %sql
select * from CCR limit 5

customer_id,acquisition_date,start_of_period_date,end_of_period_date,start_of_period_status,end_of_period_status,churn_date
1,5/22/2024,3/1/2025,3/31/2025,Active,Active,null
2,7/18/2023,3/1/2025,3/31/2025,Active,Active,null
3,8/31/2023,3/1/2025,3/31/2025,Active,Active,null
4,1/16/2024,3/1/2025,3/31/2025,Active,Active,null
5,4/4/2023,3/1/2025,3/31/2025,Active,Active,null


In [0]:
%sql
select count(*) as total_cusotmers, count(case when churn_date is not null then customer_id else null end) as churn
      ,count(case when churn_date is not null then customer_id else null end)/count(*) as churn_rate_CR  
from CCR

total_cusotmers,churn,churn_rate_CR
200,22,0.11



### 6. Revenue Churn Rate

| Category | Detail |
| :--- | :--- |
| **Definition** | The percentage of recurring revenue lost from churned customers and account downgrades over a specific period. |
| **SQL Concept** | Sum the recurring revenue lost (from churn and downgrades) and divide by the total recurring revenue at the *start* of the period. |
| **Formula** | $$\text{Revenue Churn Rate} = \frac{\text{Lost Recurring Revenue in Period}}{\text{Total Recurring Revenue at Start of Period}} \times 100$$ |
| **Data Required** | **Customer ID**, **Start-of-Period MRR/ARR** (Baseline Revenue), **Lost MRR/ARR** (from Churn/Downgrades). |




In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW RCR 
USING CSV
OPTIONS (
  path "file:/Workspace/Users/olesia.tankersley@thermofisher.com/Optimization/data/RCR.csv",
  header "true",
  inferSchema "true",
  dateFormat "yyyy/MM/dd" );

In [0]:
 %sql
select * from RCR limit 5

customer_id,start_of_period_date,end_of_period_date,start_of_period_mrr,revenue_lost_churn_downgrade,revenue_change_type
1,3/1/2024,3/31/2024,1000,0.0,Retained_or_Upsell
2,3/1/2024,3/31/2024,3939,0.0,Retained_or_Upsell
3,3/1/2024,3/31/2024,1737,0.0,Retained_or_Upsell
4,3/1/2024,3/31/2024,2537,0.0,Retained_or_Upsell
5,3/1/2024,3/31/2024,3179,0.0,Retained_or_Upsell


In [0]:
%sql
select sum(start_of_period_mrr) as starting_MRR
      ,round(sum(revenue_lost_churn_downgrade),0) as revenue_lost_churn_downgrade
      ,round(sum(revenue_lost_churn_downgrade) 
      /sum(start_of_period_mrr),2) as revenue_churn_rate_RCR
from RCR

starting_MRR,revenue_lost_churn_downgrade,revenue_churn_rate_RCR
545423,63965.0,0.12



### 7. Customer Acquisition Cost (CAC)

| Category | Detail |
| :--- | :--- |
| **Definition** | The total cost (Sales and Marketing expenses) incurred by the business to acquire one new customer. |
| **SQL Concept** | Aggregate total Sales and Marketing expenses for a given period and divide by the total number of new customers acquired in that same period. (Note: Expense data often comes from a finance ledger). |
| **Formula** | $$\text{CAC} = \frac{\text{Total Sales and Marketing Expenses}}{\text{Total New Customers Acquired}}$$ |
| **Data Required** | **Total Sales and Marketing Expenses** (from Finance/GL system), **New Customer Count**, **Acquisition Date** (for new customer cohort). |


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW CAC 
USING CSV
OPTIONS (
  path "file:/Workspace/Users/olesia.tankersley@thermofisher.com/Optimization/data/CAC.csv",
  header "true",
  inferSchema "true",
  dateformat "MM/dd/yyyy" );

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW CAC_exp 
USING CSV
OPTIONS (
  path "file:/Workspace/Users/olesia.tankersley@thermofisher.com/Optimization/data/CAC_expenses.csv",
  header "true",
  inferSchema "true",
  dateformat "yyyy/MM/dd" );

In [0]:
 %sql
 --customers acquired in the month of March:
select * from CAC limit 5

customer_id,acquisition_date,initial_mrr
1,3/17/2024,134
2,3/3/2024,359
3,3/18/2024,193
4,3/16/2024,318
5,3/16/2024,214


In [0]:
 %sql
select * from CAC_exp limit 1

reporting_month_start,reporting_month_end,total_sales_expenses,total_marketing_expenses,total_acquisition_cost
3/1/2024,3/31/2024,55000,35000,90000


In [0]:
%sql
---assuming we are calculating monthly CAC 
select total_sales_expenses, total_marketing_expenses, total_acquisition_cost
      ,count(distinct customer_id) as new_customers
      ,total_acquisition_cost/count(distinct customer_id) as CAC     
from CAC_exp
cross join CAC
group by total_sales_expenses, total_marketing_expenses, total_acquisition_cost


total_sales_expenses,total_marketing_expenses,total_acquisition_cost,new_customers,CAC
55000,35000,90000,200,450.0



### 8. Retention Rate at 30, 60, 360 days

| Category | Detail |
| :--- | :--- |
| **Definition** | The percentage of a specific starting cohort of customers (e.g., all customers acquired in January) that are still active after a set number of days (N). |
| **SQL Concept** | Define a starting cohort by acquisition date. Count the customers from that cohort who still have an 'Active' status or a recent **Last Activity Date** N days later. |
| **Formula** | $$\text{Retention Rate (N days)} = \frac{\text{Customers from Cohort Still Active after N days}}{\text{Total Customers in Cohort}} \times 100$$ |
| **Data Required** | **Customer ID**, **Acquisition Date** (for defining the cohort), **Active/Status Flag** or **Last Activity Date**, **Date of Measurement** (e.g., 30, 60, or 360 days after acquisition). |


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW RRD 
USING CSV
OPTIONS (
  path "file:/Workspace/Users/olesia.tankersley@thermofisher.com/Optimization/data/RR_days.csv",
  header "true",
  inferSchema "true" );

In [0]:
 %sql
select * from RRD limit 5

customer_id,acquisition_date,subscription_date,termination_date
1,2023-09-16,2023-02-08,null
2,2023-11-28,2023-12-02,null
3,2023-11-26,2023-01-28,2024-07-22
4,2023-02-06,2023-11-17,null
5,2023-01-23,2023-02-17,null


In [0]:
%sql

select count(case when subscription_date is not null then customer_id else null end) as total_subscribed

, count(case when termination_date > subscription_date + interval 30 days or termination_date is null 
             then customer_id else null end) as subscription_cohort_retention_30_days

, count(case when termination_date > subscription_date + interval 60 days or termination_date is null 
             then customer_id else null end) as subscription_cohort_retention_60_days

, count(case when termination_date > subscription_date + interval 360 days or termination_date is null 
             then customer_id else null end) as subscription_cohort_retention_360_days

, count(case when termination_date > subscription_date + interval 30 days or termination_date is null 
             then customer_id else null end)
             /count(case when subscription_date is not null then customer_id else null end)
              as survival_rate_30_days

, count(case when termination_date > subscription_date + interval 60 days or termination_date is null 
             then customer_id else null end) 
            /count(case when subscription_date is not null then customer_id else null end)
              as survival_rate_60_days

, count(case when termination_date > subscription_date + interval 360 days or termination_date is null 
             then customer_id else null end)  
             /count(case when subscription_date is not null then customer_id else null end)
              as survival_rate_360_days          
from RRD 


total_subscribed,subscription_cohort_retention_30_days,subscription_cohort_retention_60_days,subscription_cohort_retention_360_days,survival_rate_30_days,survival_rate_60_days,survival_rate_360_days
500,480,478,400,0.96,0.956,0.8
